In [15]:
import os
import pandas as pd
from json_to_csv import convert_to_csv

### Reading GPS track data
Read the 13 `vehiclePosition` json files and convert them to csv.


__Assumption__: All the `json` and `csv` files should be inside `data` directory

In [10]:
# Current directory path
root = os.path.realpath(".")
data_dir = os.path.join(root, 'data')
# Get all the files in the root containing the word 'vehiclePosition'
input_files = [os.path.join(data_dir, x) for x in os.listdir('.') if 'vehiclePosition' in x]
# For each input files generate the filename with csv extension
output_files = [x.replace('.json', '.csv') for x in input_files]

In [12]:
for i, file in enumerate(input_files):
    convert_to_csv(file, output_files[i])

Preprocessing  D:\3_Education\07_MS\BDMA\DataMining\Project\hack-my-ride\vehiclePosition01.json
Renaming columns
Writing output
D:\3_Education\07_MS\BDMA\DataMining\Project\hack-my-ride\vehiclePosition01.csv generated success!
Preprocessing  D:\3_Education\07_MS\BDMA\DataMining\Project\hack-my-ride\vehiclePosition02.json
Renaming columns
Writing output
D:\3_Education\07_MS\BDMA\DataMining\Project\hack-my-ride\vehiclePosition02.csv generated success!
Preprocessing  D:\3_Education\07_MS\BDMA\DataMining\Project\hack-my-ride\vehiclePosition03.json
Renaming columns
Writing output
D:\3_Education\07_MS\BDMA\DataMining\Project\hack-my-ride\vehiclePosition03.csv generated success!
Preprocessing  D:\3_Education\07_MS\BDMA\DataMining\Project\hack-my-ride\vehiclePosition04.json
Renaming columns
Writing output
D:\3_Education\07_MS\BDMA\DataMining\Project\hack-my-ride\vehiclePosition04.csv generated success!
Preprocessing  D:\3_Education\07_MS\BDMA\DataMining\Project\hack-my-ride\vehiclePosition05.j

### [Optional] Merge all the `csv` files into single file

In [18]:
csvs = [pd.read_csv(f) for f in output_files]
print('Concatenating...')
df_data = pd.concat(csvs, ignore_index=True)
gps_file = os.path.join(data_dir, 'gps_track.csv')
df_data.to_csv(gps_file, index=False)

Concatenating...
